In [1]:
from PIL import Image
from flask_cors import CORS
from flask import request,redirect,Response, jsonify
from flask import Flask, render_template
import numpy as np
import pandas as pd
import json

In [2]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [3]:
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.models import load_model
from numpy import argmax
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense,InputLayer,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import regularizers

In [34]:
s_list = ['운비쌤 도와주세요~', '집에 가고싶다']
for i in s_list:
    print(i)
    for j in range(len(i)):
        cnt+=1
        print(j)
len(s_list)
#     dictionary = {cnt+=1 for string : cnt for string in s_list}
# print(dictionary)

s_list = ['운비쌤 도와주세요~', '집에 가고싶다']
dictionary = {string : 0 for string in s_list}
print(dictionary)

운비쌤 도와주세요~
0
1
2
3
4
5
6
7
8
9
집에 가고싶다
0
1
2
3
4
5
6
{'운비쌤 도와주세요~': 0, '집에 가고싶다': 0}


In [ ]:
dictionary = {cnt+=1 for string : cnt for string in s_list}
print(dictionary)

In [12]:
last_list2=[]

In [ ]:
app=Flask(__name__)
CORS(app)
@app.route("/urlmapping", methods=['GET','POST'])
def function_name(): 
    L = []
    R = []
    L2= []
    R2= []
    if request.method=='POST':
        L=request.form['left']
        R=request.form['right']
        L1=json.loads(str(L))
        R1=json.loads(str(R))
        for i in range(len(L1)):
            if L1[i]["x"]==None:
                L2.append(np.nan)
                L2.append(np.nan)
            else:
                L2.append(L1[i]["x"])            
                L2.append(L1[i]["y"])
        for i in range(len(R1)):
            if R1[i]["x"]==None:
                R2.append(np.nan)
                R2.append(np.nan)
            else:
                R2.append(R1[i]["x"])            
                R2.append(R1[i]["y"])
        
        L3 = np.asarray(L2)
        L3=L3.reshape((int(len(L3)/42),42))
        R3 = np.asarray(R2)
        R3=R3.reshape((int(len(R3)/42),42))


        dfL1=pd.DataFrame(L3)
        dfR1=pd.DataFrame(R3)


        result = pd.concat([dfL1,dfR1],axis=1)
        result = np.array(result)
        
        df = pd.DataFrame(result)
        df = df.astype(dtype='float64', errors='ignore')
        df = df.interpolate()
        df = df.fillna(0)

        
        
        
        data=np.array(df)
        data=data.reshape(1,len(df)*84)
        print(data.shape)
        data_pad=pad_sequences(data, maxlen=7980,dtype='float64')
        print(data_pad.shape)
        data_pad_reshape=data_pad.reshape(1, 95, 84)
        
        model00 = load_model('./LSTM_Model00.h5')
        model01 = load_model('./LSTM_Model01.h5')
        model02 = load_model('./LSTM_Model02.h5')       
        
        model_dict = {0 : model00, 1 : model01, 2 : model02}
        
        weights = [0.33, 0.33, 0.33]
        for index, model in enumerate(model_dict.values()):
            if index == 0:
                y_pred0 = model.predict(data_pad_reshape)
            elif index == 1:
                y_pred1 = model.predict(data_pad_reshape)
            elif index == 2:
                y_pred2 = model.predict(data_pad_reshape)
            else:
                print("Index Wrong")

        results = np.array([y_pred0, y_pred1, y_pred2])
        Y_pred = np.tensordot(results, weights, axes = ((0), (0)))
        y_pred = np.argmax(Y_pred, axis=1)         
        
#         x=np.array(pre[0])
#         max_val = np.max(x)
#         x.index(max_val)
#       print(np.where(x == max_val))
        print(Y_pred)
        print(max(Y_pred[0]))
        num2=sorted(Y_pred[0], reverse=True)[1]
        num3=sorted(Y_pred[0], reverse=True)[2]
        print(num2)
        print(num3)
        
        
        for i in range(len(Y_pred[0])):
            if Y_pred[0][i]==max(Y_pred[0]):
                x=i
                print(x)
          
        for i in range(len(Y_pred[0])):
            if Y_pred[0][i]==num2:
                max2=i
                print(max2)     
        
        for i in range(len(Y_pred[0])):
            if Y_pred[0][i]==num3:
                max3=i
                print(max3)       
                
        
        
        
        
    
        return str(x)+"/"+str(max2)+"/"+str(max3)+"/"+str(max(Y_ped[0]))+"/"+str(num2)+"/"+str(num3)
    else:
        b="hi"
        return str(b)

if __name__ == "__main__":
    app.run(host="172.30.1.10", port="8081")


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.30.1.10:8081/ (Press CTRL+C to quit)


(1, 3780)
(1, 7980)
1/1 [==============================] - 1s 743ms/step


172.30.1.10 - - [19/Aug/2022 16:09:31] "POST /urlmapping HTTP/1.1" 200 -


[[1.09496399e-06 1.19206653e-04 1.22513005e-05 3.98758601e-04
  1.05995842e-03 9.87009497e-07 3.27862243e-01 1.64063272e-05
  3.95849884e-04 5.95543411e-06 4.88451518e-07 1.44415475e-05
  6.21007042e-07 4.31915911e-06 3.70308394e-08 6.55130167e-01
  3.29078595e-05 1.62533866e-05 5.92361247e-05 1.26249940e-06
  9.53574066e-06 3.27699094e-06 7.74187661e-07 2.77465111e-07
  4.03760533e-06 3.33346268e-05 1.82807562e-05 2.82011563e-05
  1.95155872e-05 3.85917710e-05 2.36199588e-06 6.31633456e-07
  6.40854893e-07 6.42570625e-06 9.91980246e-07 8.70951262e-07
  2.69902634e-06 1.69744876e-08 1.49455262e-07 3.56308863e-07
  3.22805677e-07 4.31505487e-06 3.72091796e-07 2.32619086e-07
  9.03451669e-07 6.10209867e-08 6.18208577e-07 2.16878335e-05
  1.87758478e-06 4.61941144e-06 5.09177182e-05 2.46757381e-06
  3.67914560e-08 1.09268205e-03 2.26551926e-06 4.84033132e-10
  4.28079733e-05 8.94677552e-04 4.36810615e-06 2.21191125e-07
  6.60503471e-07 1.36454106e-06 7.28738233e-06 9.38354484e-07
  8.0380

...

[[7.95696461e-04 1.22140929e-04 4.15085590e-04 1.76915970e-05
  1.73380408e-05 5.91942657e-04 4.41940388e-04 5.93067878e-04
  5.80702396e-05 3.85089830e-06 1.17705627e-05 8.76204830e-04
  4.54627427e-05 1.82378169e-04 9.22361979e-05 1.13117509e-04
  1.18990206e-04 1.87423253e-05 6.34906498e-05 1.35383753e-04
  1.72343802e-03 4.31817171e-04 3.30526325e-05 5.79719478e-06
  4.53872929e-01 1.26470374e-05 8.95375648e-06 1.15683662e-04
  5.80215871e-05 7.77260038e-03 9.66140790e-07 4.18482038e-03
  2.37346390e-04 1.13936516e-04 4.78788041e-03 2.52076765e-04
  1.53276126e-04 9.88441187e-02 5.00708206e-04 2.99463290e-03
  3.77936737e-03 3.66536106e-04 7.14155374e-03 5.71466420e-05
  1.66472006e-04 7.01564000e-03 1.06511417e-06 4.17392521e-05
  1.27312590e-05 1.11880174e-04 3.20685200e-02 3.74372961e-02
  4.53554690e-04 6.15022162e-03 9.04866722e-05 6.97779515e-05
  5.88868531e-03 1.46442721e-04 1.80969146e-04 4.53367340e-03
  1.61637498e-03 3.67649430e-05 7.51070903e-06 4.38371618e-03
  3.7841

172.30.1.10 - - [19/Aug/2022 16:43:02] "POST /urlmapping HTTP/1.1" 200 -


[[9.66567925e-07 1.06691221e-04 1.01536930e-01 1.49360307e-05
  5.48555004e-01 2.15652529e-05 2.95412194e-02 7.57162810e-04
  1.64542526e-05 9.61017733e-07 6.15321476e-06 2.74071891e-05
  1.62643971e-06 1.89670836e-03 4.34748883e-06 6.30900918e-03
  4.30288953e-06 9.18202612e-05 2.86693076e-08 1.05239962e-01
  2.09803657e-02 9.93401332e-06 1.04053634e-07 1.69402657e-01
  5.44756248e-04 4.29403836e-08 1.99412679e-07 3.36257103e-06
  1.62854553e-06 1.71115681e-05 3.94539913e-06 9.43928286e-04
  3.54755281e-06 6.02925422e-07 3.27957132e-07 3.15633808e-06
  5.27688671e-08 2.19371280e-06 2.67350848e-08 1.57672524e-06
  1.77769329e-07 3.28202461e-07 8.48305216e-05 1.14931147e-06
  2.78600140e-06 1.31878666e-06 9.15167863e-09 3.85586002e-06
  1.37941762e-05 4.58363453e-07 7.99179851e-06 7.33310071e-06
  1.68203568e-05 5.64808443e-07 1.06260503e-04 3.85994560e-06
  9.26311616e-07 3.24870558e-03 8.19782252e-06 6.23393224e-08
  8.65852911e-06 4.81457038e-06 7.54443992e-06 5.16853212e-07
  4.3356

172.30.1.10 - - [19/Aug/2022 16:43:15] "POST /urlmapping HTTP/1.1" 200 -


[[5.83294299e-05 3.02189891e-07 3.95415814e-04 5.75193175e-05
  1.72883806e-02 5.20728143e-06 1.11976269e-02 3.74764235e-06
  1.89446035e-05 1.36784881e-07 5.91286820e-07 6.41279131e-08
  3.18067729e-08 5.10382071e-05 3.74869076e-08 4.26752086e-07
  1.04292274e-06 3.21859246e-07 2.19595834e-07 1.02503297e-04
  1.57484965e-05 1.41555316e-05 5.09041363e-08 4.41750725e-06
  9.58507347e-01 2.37687656e-08 2.41497642e-06 1.25200641e-05
  2.56913755e-04 2.64524313e-06 1.13039500e-05 2.02956685e-05
  2.78187447e-06 1.42300313e-07 2.16873282e-07 2.51396924e-07
  6.65279050e-07 2.69816831e-06 6.89073569e-08 6.47218578e-07
  6.99395124e-06 5.42413199e-07 3.69725994e-05 9.47758401e-08
  7.33067947e-08 4.43701783e-08 4.36109207e-08 3.10717385e-09
  2.37610434e-07 4.06392333e-08 8.53155160e-07 3.05368297e-06
  2.84364323e-06 6.53944741e-07 8.13552884e-07 9.11604097e-06
  2.01678978e-06 5.65755203e-04 2.81594909e-05 1.00501564e-07
  1.97017023e-06 2.48237549e-07 1.75773399e-07 3.62081826e-08
  7.7092

In [26]:
max(pre[0])

NameError: name 'pre' is not defined

In [4]:
print(L[0]["x"])

NameError: name 'L' is not defined

In [ ]:
L=[]
for i in range(len(json_data)):
    L.append(L[i]["x"])
    L.append(L[i]["y"])
    L.append(L[i]["z"])

In [2]:
import cv2 
import mediapipe as mp  
from PIL import ImageFont, ImageDraw, Image
import numpy as np      

s_list =[]
def video():
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    mp_drawing_styles = mp.solutions.drawing_styles

    # For webcam input:
    cap = cv2.VideoCapture(0)


    with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    
        while cap.isOpened():            
            success, image = cap.read()

            if not success:
                print("Ignoring empty camera frame.")
                break

          # If loading a video, use 'break' instead of 'continue'.
                continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = hands.process(image)


            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image_height, image_width, _ = image.shape

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:


                # 엄지를 제외한 나머지 4개 손가락의 마디 위치 관계를 확인하여 플래그 변수를 설정합니다. 손가락을 일자로 편 상태인지 확인합니다.
                    thumb_finger_state = 0
                    if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y * image_height:
                            if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image_height:
                                thumb_finger_state = 1

                    index_finger_state = 0
                    if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y * image_height:
                            if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height:
                                index_finger_state = 1

                    middle_finger_state = 0
                    if hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y * image_height:
                            if hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * image_height:
                                middle_finger_state = 1

                    ring_finger_state = 0
                    if hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y * image_height:
                            if hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y * image_height:
                                ring_finger_state = 1

                    pinky_finger_state = 0
                    if hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y * image_height:
                        if hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y * image_height:
                            if hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y * image_height > hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y * image_height:
                                pinky_finger_state = 1


                    # 손가락 위치 확인한 값을 사용하여 가위,바위,보 중 하나를 출력 해줍니다.
                    font = ImageFont.truetype("fonts/gulim.ttc", 80)
                    image = Image.fromarray(image)
                    draw = ImageDraw.Draw(image)


                    text = ""
                    if thumb_finger_state == 1 and index_finger_state == 1 and middle_finger_state == 1 and ring_finger_state == 1 and pinky_finger_state == 1:
                        text = "운비쌤 도와주세요~"
                        s_list.append(text)
                    elif thumb_finger_state == 0 and index_finger_state == 0 and middle_finger_state == 1 and ring_finger_state == 0 and pinky_finger_state == 0:
                        text = "나 안할래~"
                        s_list.append(text)
                        
                    elif thumb_finger_state == 1 and index_finger_state == 1 and middle_finger_state == 0 and ring_finger_state == 0 and pinky_finger_state == 0:
                        text = "집에 가고싶다"
                        s_list.append(text)
                       
                    elif   index_finger_state == 0 and middle_finger_state == 0 and ring_finger_state == 0 and pinky_finger_state == 0:
                        text = "스마트인재개발원"
                        s_list.append(text)
                        

                    w, h = font.getsize(text)

                    x = 10
                    y = 10
    
                    draw.rectangle((x, y, x + w, y + h), fill='black')
                    draw.text((x, y),  text, font=font, fill=(255, 255, 255))
                    image = np.array(image)
                    
                    

                    # 손가락 뼈대를 그려줍니다.
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())

            cv2.imshow('MediaPipe Hands', image)

            if cv2.waitKey(5) & 0xFF == 27:
                cap.release()
                break
    cap.release()    

In [ ]:
app=Flask(__name__)
CORS(app)
@app.route("/urlmapping", methods=['GET','POST'])
def function_name():
    if request.method=='POST':
        num1= int(request.form['id'])
        result =render_template('hand3.py')
        return result  
    else:
        a='hi'
        return str(a)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port="8081")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8081
 * Running on http://172.30.1.10:8081 (Press CTRL+C to quit)
172.30.1.10 - - [08/Aug/2022 10:54:18] "GET / HTTP/1.1" 404 -
172.30.1.10 - - [08/Aug/2022 10:54:23] "GET /urlmapping HTTP/1.1" 200 -


In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import cv2
import mediapipe as mp

def video():
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_hands = mp.solutions.hands

    # For static images:

    IMAGE_FILES = []
    with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.5) as hands:
        for idx, file in enumerate(IMAGE_FILES):
            # Read an image, flip it around y-axis for correct handedness output (see
            # above).
            image = cv2.flip(cv2.imread(file), 1)
            # Convert the BGR image to RGB before processing.
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            # Print handedness and draw hand landmarks on the image.
            print('Handedness:', results.multi_handedness)
            if not results.multi_hand_landmarks:
                continue
            image_height, image_width, _ = image.shape
            annotated_image = image.copy()
            for hand_landmarks in results.multi_hand_landmarks:
                print('hand_landmarks:', hand_landmarks)
                print(
                        f'Index finger tip coordinates: (',
                        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                )
                mp_drawing.draw_landmarks(
                        annotated_image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
            cv2.imwrite(
                    '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
            # Draw hand world landmarks.
            if not results.multi_hand_world_landmarks:
                continue
            for hand_world_landmarks in results.multi_hand_world_landmarks:
                mp_drawing.plot_landmarks(
                    hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

    # For webcam input:
    cap = cv2.VideoCapture(0)
    with mp_hands.Hands(
            model_complexity=0,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)

            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                            image,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style())
            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
                break
    cap.release()



In [3]:
video()

Ignoring empty camera frame.


In [ ]:
dic[]
for i in range(100):
    for j in list:
        dic.append(i,j)

In [15]:
s_list = ['운비쌤 도와주세요~', '집에 가고싶다']
dic={'힘들어':0}
for i,j in s_list,range(len(s_list)):
        dic[j]= i
print(dic)

{'힘들어': 0, '집에 가고싶다': '운비쌤 도와주세요~', 1: 0}


<function list.pop(index=-1, /)>

[2022-08-09 17:43:37,218] ERROR in app: Exception on /urlmapping [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2097, in make_response
    raise TypeError(
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
172.30.1.10 - - [09/Aug/2022 17:43:37] "POST /urlmapping HTTP/1.1" 500 -


In [10]:
i=0
s_list = ['운비쌤 도와주세요~', '집에 가고싶다']
dictionary = {string : for i in range(len(s_list)) for string in s_list}
print(dictionary)

SyntaxError: invalid syntax (Temp/ipykernel_12400/3802034678.py, line 3)

In [ ]:
s_list = ['운비쌤 도와주세요~', '집에 가고싶다']
dictionary = {string : for i in range(len(s_list)) for string in s_list}
print(dictionary)

In [41]:
def add_dic(key, val):
    dic.setdefault(key,val)

key = str(raw_input().strip())
if key in dic:
    dic[key] += 1
else:
    add_dic(key, 1)

NameError: name 'raw_input' is not defined

In [ ]:
value=['집','갈래','보내줘']
dic={}
j=0
for i in value:
    dic[i]=j
    j=j+1
print(dic)

In [ ]:
for i in range(len(pre[0])):
    if pre[0][i]==max(pre[0]):
        print(i)